<a href="https://colab.research.google.com/github/GraWick/Deep-Learning-Project/blob/version-felippe/ShuffleNet_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
import keras.backend as K


def _stage(tensor, nb_groups, in_channels, out_channels, repeat):
    x = _shufflenet_unit(tensor, nb_groups, in_channels, out_channels, 2)

    for _ in range(repeat):
        x = _shufflenet_unit(x, nb_groups, out_channels, out_channels, 1)

    return x


def _pw_group(tensor, nb_groups, in_channels, out_channels):
    """Pointwise grouped convolution."""
    nb_chan_per_grp = in_channels // nb_groups

    pw_convs = []
    for grp in range(nb_groups):
        x = Lambda(lambda x: x[:, :, :, nb_chan_per_grp * grp: nb_chan_per_grp * (grp + 1)])(tensor)
        grp_out_chan = int(out_channels / nb_groups + 0.5)

        pw_convs.append(
            Conv2D(grp_out_chan,
                   kernel_size=(1, 1),
                   padding='same',
                   use_bias=False,
                   strides=1)(x)
        )

    return Concatenate(axis=-1)(pw_convs)


def _shuffle(x, nb_groups):
    def shuffle_layer(x):
        _, w, h, n = K.int_shape(x)
        nb_chan_per_grp = n // nb_groups

        x = K.reshape(x, (-1, w, h, nb_chan_per_grp, nb_groups))
        x = K.permute_dimensions(x, (0, 1, 2, 4, 3)) # Transpose only grps and chs
        x = K.reshape(x, (-1, w, h, n))

        return x

    return Lambda(shuffle_layer)(x)


def _shufflenet_unit(tensor, nb_groups, in_channels, out_channels, strides, shuffle=True, bottleneck=4):
    bottleneck_channels = out_channels // bottleneck

    x = _pw_group(tensor, nb_groups, in_channels, bottleneck_channels)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if shuffle:
        x = _shuffle(x, nb_groups)

    x = DepthwiseConv2D(kernel_size=(3, 3),
                        padding='same',
                        use_bias=False,
                        strides=strides)(x)
    x = BatchNormalization()(x)


    x = _pw_group(x, nb_groups, bottleneck_channels,
                  out_channels if strides < 2 else out_channels - in_channels)
    x = BatchNormalization()(x)

    if strides < 2:
        x = Add()([tensor, x])
    else:
        avg = AveragePooling2D(pool_size=(3, 3),
                               strides=2,
                               padding='same')(tensor)

        x = Concatenate(axis=-1)([avg, x])

    x = Activation('relu')(x)

    return x


def _info(nb_groups):
    return {
        1: [24, 144, 288, 576],
        2: [24, 200, 400, 800],
        3: [24, 240, 480, 960],
        4: [24, 272, 544, 1088],
        8: [24, 384, 768, 1536]
    }[nb_groups], [None, 3, 7, 3]


def ShuffleNet(input_shape, nb_classes, include_top=True, weights=None, nb_groups=8):
    x_in = Input(shape=input_shape)

    x = Conv2D(24,
               kernel_size=(3, 3),
               strides=2,
               use_bias=False,
               padding='same')(x_in)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D(pool_size=(3, 3),
                     strides=2,
                     padding='same')(x)

    channels_list, repeat_list = _info(nb_groups)
    for i, (out_channels, repeat) in enumerate(zip(channels_list[1:], repeat_list[1:]), start=1):
        x = _stage(x, nb_groups, channels_list[i-1], out_channels, repeat)

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs=x_in, outputs=x)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [0]:
from keras.datasets import cifar10
(x_train, y_train_), (x_test, y_test_) = cifar10.load_data()

'''# Load a given dataset by name, along with the DatasetInfo
import tensorflow_datasets as tfds

data, info = tfds.load("cifar10", with_info=True)
train_data, test_data = data['train'], data['test']'''

# TODO load from drive 

# Rescaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_train.shape

170500096/170498071 [==============================] - 26s 0us/step


(50000, 32, 32, 3)

In [0]:
# one-hot encoding of labels
num_classes = 10;
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

y_train = one_hot(y_train_, num_classes)
y_test = one_hot(y_test_, num_classes)

'''# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
'''

'# convert class vectors to binary class matrices\ny_train = keras.utils.to_categorical(y_train, 10)\ny_test = keras.utils.to_categorical(y_test, 10)\n'

In [0]:
########################################
# execute the model
########################################
import keras
n_epochs = 50
batch_size = 256
input_shape = [32, 32, 3]
nb_classes = 10


model = ShuffleNet(input_shape, nb_classes, include_top=True, weights=None, nb_groups=8)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.1449 - acc: 0.3150 - val_loss: 3.0268 - val_acc: 0.2951
Epoch 2/50
50000/50000 [==============================] - 29s 577us/step - loss: 1.6397 - acc: 0.4323 - val_loss: 1.9048 - val_acc: 0.4010
Epoch 3/50
50000/50000 [==============================] - 29s 581us/step - loss: 1.4475 - acc: 0.4956 - val_loss: 1.6184 - val_acc: 0.4491
Epoch 4/50
50000/50000 [==============================] - 29s 583us/step - loss: 1.2960 - acc: 0.5421 - val_loss: 1.6472 - val_acc: 0.4729
Epoch 5/50
50000/50000 [==============================] - 29s 572us/step - loss: 1.1920 - acc: 0.5801 - val_loss: 1.6913 - val_acc: 0.4709
Epoch 6/50
50000/50000 [==============================] - 29s 588us/step - loss: 1.1008 - acc: 0.6135 - val_loss: 1.6620 - val_acc: 0.4815
Epoch 7/50
50000/50000 [==============================] - 28s 568us/step - loss: 1.0617 - acc: 0.6269 - val_loss: 1.63

In [0]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_772 (Conv2D)             (None, 16, 16, 24)   648         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_148 (BatchN (None, 16, 16, 24)   96          conv2d_772[0][0]                 
__________________________________________________________________________________________________
activation_100 (Activation)     (None, 16, 16, 24)   0           batch_normalization_148[0][0]    
__________________________________________________________________________________________________
max_poolin

In [0]:
import tensorflow as tf
import keras.backend as K


def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.


# .... Define your model here ....
print(get_flops(model))

Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
55535096


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
#TODO how to access drive / save and load files
model.save_weights('/content/gdrive/My Drive/my_model_weights.h5')

KeyboardInterrupt: ignored

In [0]:
model.save('trained_model.h5')


KeyboardInterrupt: ignored